# Quadratic QNM Visualizations

## Background

Let us consider wave perturbations to the metric subject to in going and outgoing boundary conditions. To quadratic order we will have solutions to the following equations [from this paper](https://arxiv.org/pdf/2208.07379.pdf):

$$
\mathscr{D} h^{(1)} \sim 0 \quad, \quad \mathscr{D} h^{(2)} \sim h^{(1) 2}
$$

where the first equation is the one that defines the standard set of linear QNMs $\{\omega_I\}$. If we expand the metric:

$$
h = h^{(0)} + \eta h^{(1)} + \eta^2 h^{(2)} + ...
$$

and if $\eta$ is not sufficiently small enough to suppress $h^2$ compared to $h^1$ (I think), then modes that make up $h^{(2)}$ are not ignorable. This set of Quasinormal modes are called the Quadratic Quasinormal Modes or QQNMs 

If we work through the algebra (or look at [point (1.) on pg 2 in this paper](https://arxiv.org/pdf/2208.07379.pdf)) we will find that the set of QQNMs are defined by:

$$
\textrm{QQNMs} := \{ \Re[\omega_I] \pm \Re[\omega_J] + i(\Im[\omega_I] + \Im[\omega_J]) \ \ \forall\ \omega_I, \omega_J \in \textrm{Linear QNMs}\}
$$

# Initializations

In [34]:
import qnm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ipywidgets import FloatSlider
from ipywidgets import interact, interact_manual
import mpld3


def add_dimensions(x, M, a):
    fref = 2985.668287014743; mref = 68.0;
    
def CalculateColumns(df, M):
    df['f'] = df['omega_R']/(2*np.pi)
    df['gamma'] = -df['omega_I']
    df['tau'] = 1/df['gamma']
    fref = 2985.668287014743; mref = 68.0;
    f0 = fref*mref/M
    df['f_units'] = f0*df['f']
    df['gamma_units'] = (f0)*df['gamma']
    df['tau_units'] = (1/f0)*df['tau']
    df['tau_units_ms'] = (df['tau_units'])*(10**3)
    df['f_units_Hz'] = (df['f_units'])
    return df
    
def CalculateQNMs(df, a, M):
    get_QNMs = lambda a : (lambda x: x['qnm_object'](a=a)[0])
    df['omega'] = df.apply(get_QNMs(a), axis=1)
    df['omega_R'] = np.real(df['omega'])
    df['omega_I'] = np.imag(df['omega'])
    df = CalculateColumns(df, M)
    return df.drop('qnm_object', axis=1)

## Create the list of modes

Set the parameters of $(s,l,m,n)$ that we want to restrict to. Here we also choose the 

In [35]:
s = -2;
l_s = [2, 3, 4]
n_s = [0, 1, 2, 3]

M = 68;
a = 0.7;

## Define the functions to make plots

In [36]:
modes = [];
for l in l_s:
    for m in range(-l, l+1,1):
        for n in n_s:
            mode_params = {'s':s, 'l': l, 'm': m, 'n': n}
            mode_params.update({'qnm_object': qnm.modes_cache(**mode_params)})
            mode_params.update({'mode': "(" + ",".join([str(val) for val in [l,m,n]]) + ")"})
            modes.append(mode_params)
            
df = pd.DataFrame(modes)

In [37]:
is_corotating = lambda x: x['l'] == x['m']
is_counterrotating = lambda x: x['l'] == -x['m']
is_head_on = lambda x: x['m'] == 0

def calculate_linear_and_quadratic(df, a, M):
    all_modes = CalculateQNMs(df, a, M)
    selected_modes = all_modes.copy()[all_modes.apply(is_corotating, axis=1)]
    selected_modes['order'] = 'Linear'
    other_selected_modes = selected_modes.copy()
    other_selected_modes['mode'] = other_selected_modes['mode'].apply(lambda x: x + "*")
    other_selected_modes['omega_R'] = -other_selected_modes['omega_R']
    other_selected_modes['f'] = -other_selected_modes['f']
    other_selected_modes['f_units'] = -other_selected_modes['f_units']
    other_selected_modes['f_units_Hz'] = -other_selected_modes['f_units_Hz']
    selected_modes = pd.concat([selected_modes, other_selected_modes])


    df2 = selected_modes.copy()
    make_dicts = lambda a,b: [{'mode': f"{a['mode']} x {b['mode']}",
                               'omega_R': a['omega_R'] + b['omega_R'],
                               'omega_I': a['omega_I'] + b['omega_I'],
                               'l1': a['l'], 'l2':b['l'], 
                               'm1': a['m'], 'm2':b['m'],
                               'n1': a['n'], 'n2':b['n'], 'order': 'Quadratic'}]

    second_order = np.array([[make_dicts(a,b) for i,a in df2.iterrows()] for j,b in df2.copy().iterrows()]).flatten()
    second_order = pd.DataFrame(list(second_order))
    second_order = CalculateColumns(second_order, M)
    return selected_modes, second_order

# Visualizations
## $\omega_R$ vs $-\omega_I$

In [38]:
import matplotlib
mpld3.enable_notebook() 
matplotlib.rcParams.update({'font.size': 20})

M = 68;
a = 0.7;

selected_modes, second_order = calculate_linear_and_quadratic(df, a, M)

fig, ax = plt.subplots(1, figsize=(10,8), dpi=120)

col1 = 'omega_R'
col2 = 'gamma'

scatter_1 = ax.scatter(selected_modes[col1], selected_modes[col2],c='r', s=40, label="Linear", alpha=1, linewidths=1)
scatter_2 = ax.scatter(second_order[col1], second_order[col2],c='b',s=40, label="Quadratic", alpha=0.6)

labels_1 = list(selected_modes['mode'].values)
tooltip_1 = mpld3.plugins.PointLabelTooltip(scatter_1, labels=labels_1)
mpld3.plugins.connect(fig, tooltip_1)

labels_2 = list(second_order['mode'].values)
tooltip_2 = mpld3.plugins.PointLabelTooltip(scatter_2, labels=labels_2)
mpld3.plugins.connect(fig, tooltip_2)

plt.legend(loc='upper left', borderpad=0.01)
plt.grid(alpha=0.3)

ax.set_xlim((-1.5,1.5))
ax.set_ylim((0,0.8))
ax.set_xlabel("omega_R")
ax.set_ylabel("-omega_I")
ax.set_title("All Co-rotating Quasinormal Modes (for all orders) \n are a lattice generated by the Linear Modes", fontsize=20)

mpld3.display()

## $f$ vs $\tau$
Then lets plot $f$ in Hz vs $\tau$ in ms for a given Mass and spin

In [39]:
import matplotlib
mpld3.enable_notebook() 
matplotlib.rcParams.update({'font.size': 18})

M = 68;
a = 0.7;

selected_modes, second_order = calculate_linear_and_quadratic(df, a, M)

fig, ax = plt.subplots(1, figsize=(10,7), dpi=120)

col1 = 'f_units_Hz'
col2 = 'tau_units_ms'

scatter_1 = ax.scatter(selected_modes[col1], selected_modes[col2],c='r', s=40, label="Linear", alpha=1, linewidths=1)
scatter_2 = ax.scatter(second_order[col1], second_order[col2],c='b',s=40, label="Quadratic", alpha=0.6)

labels_1 = [f"{x['mode']} [{np.round(x[col1],2)} Hz, {np.round(x[col2],2)} ms]" for i,x in selected_modes.iterrows()]
tooltip_1 = mpld3.plugins.PointLabelTooltip(scatter_1, labels=labels_1)
mpld3.plugins.connect(fig, tooltip_1)

labels_2 = [f"{x['mode']} [{np.round(x[col1],2)} Hz, {np.round(x[col2],2)} ms]" for i,x in second_order.iterrows()]
tooltip_2 = mpld3.plugins.PointLabelTooltip(scatter_2, labels=labels_2)
mpld3.plugins.connect(fig, tooltip_2)

plt.legend(loc='upper left', borderpad=0.01)
plt.grid(alpha=0.3)

ax.set_xlim((-1024,1024))
ax.set_ylim((0,5))
ax.set_xlabel("f (Hz)")
ax.set_ylabel(r"tau" + " (ms)")
ax.set_title(f"All Co-rotating Quasinormal Modes (for all orders) \n are a lattice generated by the Linear Modes with M = {M} and a = {a}", fontsize=20)

mpld3.display()

# Interactive Plot

In [40]:
@interact
def create_plot(a=FloatSlider(min=0, max=1, step=0.001, value=a), M=M):
    selected_modes, second_order = calculate_linear_and_quadratic(df, a, M)

    fig, ax = plt.subplots(1, figsize=(10,7), dpi=100)

    col1 = 'f_units_Hz'
    col2 = 'tau_units_ms'

    scatter_1 = ax.scatter(selected_modes[col1], selected_modes[col2],c='r', s=40, label="Linear", alpha=1, linewidths=1)
    scatter_2 = ax.scatter(second_order[col1], second_order[col2],c='b',s=40, label="Quadratic", alpha=0.6)

    #labels_1 = [f"{x['mode']} [{np.round(x[col1],2)} Hz, {np.round(x[col2],2)} ms]" for i,x in selected_modes.iterrows()]
    #tooltip_1 = mpld3.plugins.PointLabelTooltip(scatter_1, labels=labels_1)
    #mpld3.plugins.connect(fig, tooltip_1)

    #labels_2 = [f"{x['mode']} [{np.round(x[col1],2)} Hz, {np.round(x[col2],2)} ms]" for i,x in second_order.iterrows()]
    #tooltip_2 = mpld3.plugins.PointLabelTooltip(scatter_2, labels=labels_2)
    #mpld3.plugins.connect(fig, tooltip_2)

    plt.legend(loc='upper left', borderpad=0.01)
    plt.grid(alpha=0.3)

    ax.set_xlim((-1024,1024))
    ax.set_ylim((0,5))
    ax.set_xlabel(r"f (Hz)")
    ax.set_ylabel(r"tau" + " (ms)")
    ax.set_title("All Co-rotating Quasinormal Modes (for all orders) \n are a lattice generated by the Linear Modes", fontsize=20)

    #mpld3.display()
    plt.show()

interactive(children=(FloatSlider(value=0.7, description='a', max=1.0, step=0.001), IntSlider(value=68, descri…